In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import os

## 1. Extracción de datos

Para saber cuales han sido tanto los pilotos ganadores de los campeonatos como los equipos ganadores he usado una tabla de la siguiente página web. La tabla contiene un total de 75 filas, desde 1950 hasta 2024. La tabla contiene todos los campeonatos de F1 que se han disputado. La tabla contiene datos como el nombre del piloto ganador, el equipo con el que corrió, cuantas poles consiguió, vueltas rápidas y en qué carrera consiguió el campeonato matemáticamente.

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_Formula_One_World_Drivers%27_Champions"
response = requests.get(url)
print(response.status_code) #Si da 200 ha funcionado. Si da 400 error propio y 500 error del servidor.

200


In [4]:
# Extraer tabla de datos de los ganadores de cada año.
url = "https://en.wikipedia.org/wiki/List_of_Formula_One_World_Drivers%27_Champions"

soup = BeautifulSoup(response.text, "html.parser")
tabla = soup.find("table", {"class": "wikitable sortable"})
filas = tabla.find_all("tr")
data = []

for fila in filas[1:]:  # Ignora la primera fila que es el encabezado
    cols = fila.find_all("td")
    
    if len(cols) == 16:
        year = cols[0].text.strip()
        driver = cols[1].text.strip()
        age = cols[2].text.strip()
        team = cols[3].text.strip()
        engine = cols[4].text.strip()
        tyres = cols[5].text.strip()
        pole = cols[6].text.strip()
        wins = cols[7].text.strip()
        podiums = cols[8].text.strip()
        fastest_laps = cols[9].text.strip()
        points = cols[10].text.strip()
        percent_points = cols[11].text.strip()
        clinched = cols[12].text.strip()
        rounds_remaining = cols[13].text.strip()
        margin = cols[14].text.strip()
        percent_margin = cols[15].text.strip()
        
        data.append([year, driver, age, team, engine, tyres, pole, wins, 
                     podiums, fastest_laps, points, percent_points, clinched, 
                     rounds_remaining, margin, percent_margin])

df_temporadas_f1 = pd.DataFrame(data, columns=[
    "year", "Driver", "Age", "Team", "Engine", "Tyres", "Pole", 
    "Wins", "Podiums", "Fastest Laps", "Points", "% Points", "Clinched", 
    "# of Rounds Remaining", "Margin", "% Margin"
])

# Eliminar los corchetes y contenido de las columnas que tienen corchetes para limpiar los datos.
for column in df_temporadas_f1.columns:
    for index in range(len(df_temporadas_f1)):
        df_temporadas_f1.at[index, column] = re.sub(r'\[.*?\]', '', str(df_temporadas_f1.at[index, column]))

df_temporadas_f1["year"] = pd.to_numeric(df_temporadas_f1["year"])
df_temporadas_f1.loc[df_temporadas_f1["Tyres"] == "F P", "Tyres"] = "F"
df_temporadas_f1.loc[df_temporadas_f1["Tyres"] == "M G", "Tyres"] = "G"
df_temporadas_f1

,year,Driver,Age,Team,Engine,Tyres,Pole,Wins,Podiums,Fastest Laps,Points,% Points,Clinched,# of Rounds Remaining,Margin,% Margin
0,1950,Giuseppe Farina,44,Alfa Romeo,Alfa Romeo,P,2,3,3,3,30,83.333 (47.619),Round 7 of 7,0,3,10.000
1,1951,Juan Manuel Fangio,40,Alfa Romeo,Alfa Romeo,P,4,3,5,5,31,86.111 (51.389),Round 8 of 8,0,6,19.355
2,1952,Alberto Ascari,34,Ferrari,Ferrari,F,5,6,6,6,36,100.000 (74.306),Round 6 of 8,2,12,33.333
3,1953,Alberto Ascari,35,Ferrari,Ferrari,P,6,5,5,4,34.5,95.833 (57.407),Round 8 of 9,1,6.5,18.841
4,1954,Juan Manuel Fangio,43,Maserati,Maserati,P,5,6,7,3,42,93.333 (70.547),Round 7 of 9,2,16.857,40.136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2020,Lewis Hamilton,35,Mercedes,Mercedes,P,10,11,14,6,347,78.507,Round 14 of 17,3,124,35.735
71,2021,Max Verstappen,24,Red Bull,Honda,P,10,10,18,6,395.5,69.692,Round 22 of 22,0,8,2.023
72,2022,Max Verstappen,25,Red Bull,RBPT,P,7,15,17,5,454,76.174,Round 18 of 22,4,146,32.159
73,2023,Max Verstappen,26,Red Bull,Honda RBPT,P,12,19,21,9,575,92.742,Round 17 of 22,5,290,50.435


DF TEMPORADAS F1
|Variable|Descripción|Tipo_de_variable|Importancia Inicial|Notas|
|-|-|-|-|-|
|year|Año de la temporada||||
|Driver|Nombre del piloto||||
|Age|Edad con la que ganó el piloto el campeonato||||
|Team|Equipó con el que ganó el piloto|||Los equipos que aparecen no son los equipos ganadores en todos los casos|
|Engine|Marca del motor con el que corrieron||||
|Tyres|Marca de la rueda: P = Pirelli, F = Firestone||||
|Pole|Número total de poles consegidas por el piloto esa temporada||||
|Wins|Numero total de victorias conseguidas por el piloto esa temporada||||
|Podiums|Número total de podiums conseguidos por el piloto esa temporada|||Se considera podium terminar entre los 3 primeros|
|Fastest Laps|Número total de vueltas rápidas realizadas por el piloto en la temporada||||
|Points|Número total de puntos conseguidos por el piloto||||
|%points|Porcentaje de puntos conseguidos por el piloto con respecto al total de puntos del equipo|||Como solo hay dos pilotos el resto del porcentaje pertenece al otro piloto del equipo|
|Clinched|En qué carrera consiguió matemáticamente el piloto el título mundial||||
|Rounds remainings|Número de carreras que quedaban para terminar la temporada cuando el piloto consiguió el título mundial||||
|Margin|Diferencia de puntos con respecto al segundo del campeonato||||
|% margin|Porcentaje de punto de diferencia del primero al segundo||||

In [5]:
# Cambio del valores de las columnas para hacerlo más manejable a la hora de analizarlo.
columnas_cambio_tipo = ["Age","Pole","Wins","Podiums","Fastest Laps","# of Rounds Remaining"]
df_temporadas_f1[columnas_cambio_tipo] = df_temporadas_f1[columnas_cambio_tipo].astype(int)
df_temporadas_f1["Points"] = df_temporadas_f1["Points"].astype(float)
df_temporadas_f1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   75 non-null     int64  
 1   Driver                 75 non-null     object 
 2   Age                    75 non-null     int64  
 3   Team                   75 non-null     object 
 4   Engine                 75 non-null     object 
 5   Tyres                  75 non-null     object 
 6   Pole                   75 non-null     int64  
 7   Wins                   75 non-null     int64  
 8   Podiums                75 non-null     int64  
 9   Fastest Laps           75 non-null     int64  
 10  Points                 75 non-null     float64
 11  % Points               75 non-null     object 
 12  Clinched               75 non-null     object 
 13  # of Rounds Remaining  75 non-null     int64  
 14  Margin                 75 non-null     object 
 15  % Margin

In [13]:
def card_tipo(df, umbral_categorica = 16, umbral_continua = 30):
    df_temp = pd.DataFrame([df.nunique(), df.nunique() / len(df) * 100, df.dtypes])
    df_temp = df_temp.T
    df_temp = df_temp.rename(columns = {0: "Card", 1: "%_Card", 2: "Tipo"})

    df_temp.loc[df_temp.Card == 1, "%_Card"] = 0.00

    df_temp["Tipo sugerido"] = "Categorica"
    df_temp.loc[df_temp["Card"] == 2, "Tipo sugerido"] = "Binaria"
    df_temp.loc[df_temp["Card"] >= umbral_categorica, "Tipo sugerido"] = "Numerica discreta"
    df_temp.loc[df_temp["%_Card"] >= umbral_continua, "Tipo sugerido"] = "Numerica continua"
    
    return df_temp
card_tipo(df_temporadas_f1)

,Card,%_Card,Tipo,Tipo sugerido
year,75,100.0,int64,Numerica continua
Driver,34,45.333333,object,Numerica continua
Age,21,28.0,int64,Numerica discreta
Team,16,21.333333,object,Numerica discreta
Engine,14,18.666667,object,Categorica
Tyres,8,10.666667,object,Categorica
Pole,16,21.333333,int64,Numerica discreta
Wins,14,18.666667,int64,Categorica
Podiums,16,21.333333,int64,Numerica discreta
Fastest Laps,11,14.666667,int64,Categorica


In [12]:
# Crear DataFrame con la información básica
df_tipificacion = pd.DataFrame([df_temporadas_f1.nunique(), df_temporadas_f1.nunique()/len(df_temporadas_f1) * 100, df_temporadas_f1.dtypes]).T.rename(columns={0: "Card", 1: "%_Card", 2: "Tipo"})

# Inicializamos como 'Categorica' por defecto
df_tipificacion["Clasificada_como"] = "Categorica"

# Clasificación de columnas binarias
df_tipificacion.loc[df_tipificacion.Card == 2, "Clasificada_como"] = "Binaria"

# Clasificación de columnas numéricas discretas (más de 11 valores únicos)
df_tipificacion.loc[df_tipificacion["Card"] > 11, "Clasificada_como"] = "Numerica Discreta"

# Clasificación de columnas numéricas continuas (más del 45% de valores únicos)
df_tipificacion.loc[(df_tipificacion["%_Card"] > 45) & (df_tipificacion["Tipo"].isin(['int64', 'float64'])), "Clasificada_como"] = "Numerica Continua"

# Mostrar el DataFrame final
df_tipificacion


,Card,%_Card,Tipo,Clasificada_como
year,75,100.0,int64,Numerica Continua
Driver,34,45.333333,object,Numerica Discreta
Age,21,28.0,int64,Numerica Discreta
Team,16,21.333333,object,Numerica Discreta
Engine,14,18.666667,object,Numerica Discreta
Tyres,8,10.666667,object,Categorica
Pole,16,21.333333,int64,Numerica Discreta
Wins,14,18.666667,int64,Numerica Discreta
Podiums,16,21.333333,int64,Numerica Discreta
Fastest Laps,11,14.666667,int64,Categorica


El siguiente df contiene datos de todas las carreras disputadas en los 75 años de formula 1. Para poder realizar este df he cogido un dataset de Kaggle que contenía 14 archivos con distintos tipos de datos. Para poder tener todos los datos en un solo df he combinado todos los archivos. Como algunas columnas eran repetidas debido a que contenian claves primarias y foraneas, he incluido que algunas columnas tengan columnas y no de problemas a la hora de realizar el merge de las tablas.

In [7]:
circuits = pd.read_csv("../data/f1_data/circuits.csv")
constructors = pd.read_csv("../data/f1_data/constructors.csv")
constructor_results = pd.read_csv("../data/f1_data/constructor_results.csv")
constructor_standings = pd.read_csv("../data/f1_data/constructor_standings.csv")
drivers = pd.read_csv("../data/f1_data/drivers.csv")
driver_standings = pd.read_csv("../data/f1_data/driver_standings.csv")
lap_times = pd.read_csv("../data/f1_data/lap_times.csv")
pit_stops = pd.read_csv("../data/f1_data/pit_stops.csv")
qualifying = pd.read_csv("../data/f1_data/qualifying.csv")
races = pd.read_csv("../data/f1_data/races.csv")
results = pd.read_csv("../data/f1_data/results.csv")
seasons = pd.read_csv("../data/f1_data/seasons.csv")
status = pd.read_csv("../data/f1_data/status.csv")

df_combinado = pd.merge(results, races, on='raceId', how='left')

df_combinado = pd.merge(df_combinado, drivers, on='driverId', how='left')

if 'constructorId' in constructors.columns:
    df_combinado = pd.merge(df_combinado, constructors, on='constructorId', how='left')

if 'constructorId' in constructor_results.columns:
    df_combinado = pd.merge(df_combinado, constructor_results, on=['raceId', 'constructorId'], how='left')

if 'constructorId' in constructor_standings.columns:
    df_combinado = pd.merge(df_combinado, constructor_standings, on=['raceId', 'constructorId'], how='left')

if 'raceId' in qualifying.columns and 'driverId' in qualifying.columns:
    df_combinado = pd.merge(df_combinado, qualifying, on=['raceId', 'driverId'], how='left', suffixes=('', '_qualifying'))

df_combinado = df_combinado.loc[:, ~df_combinado.columns.str.endswith('_qualifying')]

if 'raceId' in driver_standings.columns and 'driverId' in driver_standings.columns:
    df_combinado = pd.merge(df_combinado, driver_standings, on=['raceId', 'driverId'], how='left', suffixes=('', '_driver_standings'))

# Eliminar las columnas duplicadas si ya existen en el DataFrame
df_combinado = df_combinado.loc[:, ~df_combinado.columns.str.endswith('_driver_standings')]

if 'year' in seasons.columns:
    df_combinado = pd.merge(df_combinado, seasons, on='year', how='left', suffixes=('', '_seasons'))

    df_combinado = df_combinado.loc[:, ~df_combinado.columns.str.endswith('_seasons')]
    
if 'statusId' in status.columns:
    df_combinado = pd.merge(df_combinado, status, on='statusId', how='left')

df_combinado = df_combinado.drop_duplicates()
pd.set_option("display.max_columns", None)

Una vez juntados todos los datos en un solo df, realizamos una limpieza de los datos y de los nombres de las columnas para facilitar su entendimiento.

In [8]:
# Eliminar colunmas que no nos interesan.
df_combinado.drop(columns=["url_x","url_y","url","quali_date","quali_time","sprint_date","sprint_time",
                          "name_y","surname","fp1_date","fp1_time","fp2_date","fp2_time",
                          "fp3_date","fp3_time","milliseconds","time_y","positionText_x","number_y",
                          "positionText_y","driverId","qualifyId","number","statusId","resultId","constructorId",
                          "positionText","rank","driverStandingsId","status_x","constructorResultsId",
                          "constructorStandingsId","position","circuitId"], inplace = True)

# Renombrar columnas para entenderlas.
df_combinado.rename(columns={"nationality_y":"team_nation","nationality_x":"driver_nation","constructorRef":"team",
                            "date":"race_date","name_x":"grand_prix_name","time_x":"driver_race_time",
                            "position_x":"driver_race_position","number_x":"car_number","points_x":"driver_points",
                            "wins":"team_wins","points_y":"team_points","position_y":"team_position",
                            "status_y":"driver_race_status","points":"total_points"}, inplace=True)
df_combinado

,raceId,car_number,grid,driver_race_position,positionOrder,driver_points,laps,driver_race_time,fastestLap,fastestLapTime,fastestLapSpeed,year,round,grand_prix_name,race_date,driverRef,code,forename,dob,driver_nation,team,team_nation,team_points,total_points,team_position,team_wins,q1,q2,q3,driver_race_status
0,18,22,1,1,1,10.0,58,1:34:50.616,39,1:27.452,218.300,2008,1,Australian Grand Prix,2008-03-16,hamilton,HAM,Lewis,1985-01-07,British,mclaren,British,14.0,14.0,1.0,1.0,1:26.572,1:25.187,1:26.714,Finished
1,18,3,5,2,2,8.0,58,+5.478,41,1:27.739,217.586,2008,1,Australian Grand Prix,2008-03-16,heidfeld,HEI,Nick,1977-05-10,German,bmw_sauber,German,8.0,8.0,3.0,0.0,1:25.960,1:25.518,1:27.236,Finished
2,18,7,7,3,3,6.0,58,+8.163,41,1:28.090,216.719,2008,1,Australian Grand Prix,2008-03-16,rosberg,ROS,Nico,1985-06-27,German,williams,British,9.0,9.0,2.0,0.0,1:26.295,1:26.059,1:28.687,Finished
3,18,5,11,4,4,5.0,58,+17.181,58,1:28.603,215.464,2008,1,Australian Grand Prix,2008-03-16,alonso,ALO,Fernando,1981-07-29,Spanish,renault,French,5.0,5.0,4.0,0.0,1:26.907,1:26.188,\N,Finished
4,18,23,3,5,5,4.0,58,+18.014,43,1:27.418,218.385,2008,1,Australian Grand Prix,2008-03-16,kovalainen,KOV,Heikki,1981-10-19,Finnish,mclaren,British,14.0,14.0,1.0,1.0,1:25.664,1:25.452,1:27.079,Finished
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26514,1132,31,18,16,16,0.0,50,\N,46,1:30.875,233.371,2024,12,British Grand Prix,2024-07-07,ocon,OCO,Esteban,1996-09-17,French,alpine,French,0.0,9.0,8.0,0.0,1:34.557,\N,\N,+2 Laps
26515,1132,11,0,17,17,0.0,50,\N,50,1:29.707,236.409,2024,12,British Grand Prix,2024-07-07,perez,PER,Sergio,1990-01-26,Mexican,red_bull,Austrian,18.0,373.0,1.0,7.0,1:38.348,\N,\N,+2 Laps
26516,1132,24,14,18,18,0.0,50,\N,43,1:31.014,233.014,2024,12,British Grand Prix,2024-07-07,zhou,ZHO,Guanyu,1999-05-30,Chinese,sauber,Swiss,0.0,0.0,10.0,0.0,1:31.190,1:27.867,\N,+2 Laps
26517,1132,63,1,\N,19,0.0,33,\N,3,1:31.298,232.289,2024,12,British Grand Prix,2024-07-07,russell,RUS,George,1998-02-15,British,mercedes,German,25.0,221.0,4.0,2.0,1:30.106,1:26.723,1:25.819,Water pressure


He pensado en crear un database en el que almacenar todos los archivos de el link de kaggle para así conectar las claves primarias con las foraneas de las otras tablas, pero puede que no sea la mejor solución ya que para hacer las claves hay que borrar las tablas y crearlas de nuevo introduciendo los datos.

DF COMBINADO
|Variable|Descripción|Tipo_de_variable|Importancia inicial|Notas|
|-|-|-|-|-|
|raceId|Id de la carrera||||
|car_number|Número del coche||||
|grid|Posición de salida en la carrera||||
|driver_race_position|Posición final del piloto en la carrera||||
|positionOrder|Orden de la posiciones finales de la carrera|||Si el piloto no ha podido terminar la carrera, se cuenta el orden en el que ha terminado|
|driver_points|Puntos conseguidos en la carrera por el piloto||||
|laps|Número total de vueltas realizadas por el piloto en la carrera||||
|driver_race_time|Tiempo total de la carrera del piloto|||Si el valor que aparece es un número con +, es porque se toma como referencia el tiempo del primero en llegar a meta|
|fastestLap|Vuelta más rápida del piloto en carrera||||
|fastestLapTime|Tiempo de la vuelta más rápida||||
|fastestLapSpeed|Velocidad media de la vuelta más rápida||||
|year|Año de la carrera||||
|round|Orden de la carrera en el calendario de la temporada. El 1 es la primera carrera, el 2 la segunda, ...||||
|grand_prix_name|Nombre del GP|||Normalmente es el lugar en el que se realiza la carrera pero hay alguno con "Europe GP" porque hay dos carreras en el mismo país|
|race_date|Fecha (Año/Mes/Día) en la que se disputó la carrera||||
|driverRef|Apellido del piloto||||
|code|Código usado para mostrar el nombre del piloto en pantalla|||Código que también usan los mecánicos de los equipos|
|forename|Nombre del piloto||||
|dob|Fecha (Año/Mes/Día) del nacimiento del piloto||||
|driver_nation|Nacionalidad del piloto||||
|team|Equipo para el que corre el piloto||||
|team_nation|Nacionalidad del equipo||||
|team_points|Puntos conseguidos para el equipo por los dos pilotos en el momento en el que finaliza la carrera|||No son los puntos totales que llevan esa temporada|
|total_points|Puntos total obtenidos por el equipo en el momento que finaliza la carrera||||
|team_positon|Posición del equipo en el campeonato de constructores en el momento de finalizar la carrera||||
|team_wins|Victorias totales conseguidas entre los dos pilotos del equipo||||
|q1|Tiempo de la mejor vuelta del piloto en q1|||Si el valor que aparece es `\N` es porque el piloto no pudo superar el tiempo de corte|
|q2|Tiempo de la mejor vuelta del piloto en q2|||Si el valor que aparece es `\N` es porque el piloto no pudo superar el tiempo de corte|
|q3|Tiempo de la mejor vuelta del piloto en 13|||Es la última fase de la quali|
|driver_race_status|Estado final del piloto en la carrera: Terminada, Colisión, Descalificado, Fallo de motor, ...||||

In [9]:
df_combinado.to_csv("df_combinado.csv", index = False)

In [10]:
df_tipificacion = pd.DataFrame([df_combinado.nunique(), df_combinado.nunique()/len(df_combinado) * 100, df_combinado.dtypes]).T.rename(columns = {0: "Card",\
                                                                                                                                 1: "%_Card", 2: "Tipo"})

df_tipificacion["Clasificada_como"] = "Categorica" # Partiendo de que casi todas parecen categóricas
df_tipificacion.loc[df_tipificacion.Card == 2, "Clasificada_como"] = "Binaria"
df_tipificacion.loc[df_tipificacion["Card"] > 11, "Clasificada_como"] ="Numerica Discreta"
df_tipificacion.loc[df_tipificacion["%_Card"] > 5, "Clasificada_como"] = "Numerica Continua"
df_tipificacion

,Card,%_Card,Tipo,Clasificada_como
raceId,1113,4.196991,int64,Numerica Discreta
car_number,130,0.490215,object,Numerica Discreta
grid,35,0.131981,int64,Numerica Discreta
driver_race_position,34,0.12821,object,Numerica Discreta
positionOrder,39,0.147064,int64,Numerica Discreta
driver_points,39,0.147064,float64,Numerica Discreta
laps,172,0.648592,int64,Numerica Discreta
driver_race_time,7272,27.421848,object,Numerica Continua
fastestLap,81,0.305441,object,Numerica Discreta
fastestLapTime,7298,27.519891,object,Numerica Continua
